In [4]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from urllib.request import urlopen
import requests
import ssl
import json
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Cargar datos de URL y convertir a DataFrame
@st.cache_data # este decorador permite almacenar en caché los datos para evitar recargas innecesarias
def load_data(url, type='csv', file_name='temp'):
    # Cabeceras para la solicitud
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9",
        "Accept-Language": "en-US,en;q=0.9",
    }

    # Realizar la solicitud
    req = urllib.request.Request(url, headers=headers, method='GET')

    # Descargar el archivo
    with urlopen(req) as response, open(file_name, 'wb') as out_file:
        out_file.write(response.read())

    # Leer el archivo según su tipo
    if type == 'csv':
        return pd.read_csv(file_name)
    elif type == 'xlsx':
        return pd.read_excel(file_name)
    elif type == 'json':
        with open(file_name, 'r') as file:
            data = json.load(file)
        return pd.DataFrame(data)
    else:
        raise ValueError("Unsupported file type")

2025-04-20 21:12:36.588 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [20]:
url = "https://www.datosabiertos.gob.pe/sites/default/files/Catalogo1960_2023.xlsx"
df = load_data(url, type='xlsx', file_name='earthquakes.xlsx')
# Imprimir dataframe en consola
df.head()

,ID,FECHA_UTC,HORA_UTC,LATITUD,LONGITUD,PROFUNDIDAD,MAGNITUD,FECHA_CORTE
0,0,19600113,154034,-16.145,-72.144,60,7.5,20223006
1,1,19600115,93024,-15.000,-75.000,70,7.0,20223006
2,2,19600117,25758,-14.500,-74.500,150,6.4,20223006
3,3,19600123,33732,-12.500,-68.500,300,5.8,20223006
4,4,19600130,50724,-5.500,-77.500,100,5.7,20223006


In [21]:
# Eliminar columnas innecesarias
df.drop(columns=['ID', 'FECHA_CORTE'], inplace=True)

# Renombrar columnas
df.rename(columns={
    'FECHA_UTC': 'Fecha',
    'HORA_UTC': 'Hora',
    'LATITUD': 'Latitud',
    'LONGITUD': 'Longitud',
    'MAGNITUD': 'Magnitud',
    'PROFUNDIDAD': 'Profundidad',
}, inplace=True)

# Convertir columnas a tipos adecuados
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce', format='%Y%m%d')
df['Hora'] = pd.to_datetime(df['Hora'], errors='coerce', format='%H%M%S').dt.time
df['Latitud'] = pd.to_numeric(df['Latitud'], errors='coerce')
df['Longitud'] = pd.to_numeric(df['Longitud'], errors='coerce')
df['Magnitud'] = pd.to_numeric(df['Magnitud'], errors='coerce')
df['Profundidad'] = pd.to_numeric(df['Profundidad'], errors='coerce')

In [24]:
df.head()

,Fecha,Hora,Latitud,Longitud,Profundidad,Magnitud
0,1960-01-13,15:40:34,-16.145,-72.144,60,7.5
1,1960-01-15,09:30:24,-15.000,-75.000,70,7.0
2,1960-01-17,02:57:58,-14.500,-74.500,150,6.4
3,1960-01-23,03:37:32,-12.500,-68.500,300,5.8
4,1960-01-30,05:07:24,-5.500,-77.500,100,5.7
